In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr



In [ ]:
##The old fashioned setup

##Loading the enviroment variables
load_dotenv(override=True)
#Creating the OpenAI client
openai=OpenAI()


## PushOver Related code

(pushover desc)

### Loading Enviroment Variables

In [ ]:
#Loading Pushover related  variables
pushover_site_url="https://api.pushover.net/1/messages.json"

pushover_user_key = os.getenv("PUSHOVER_USER_KEY")

pushover_api_key = os.getenv("PUSHOVER_API_KEY")


In [ ]:
#
def pushover_notification(message):
    """Sends a pushover notification to the user, a  super simple POST request to the Pushover API"""
    print(f"Sending push notification: {message}")

    payload={"user": pushover_user_key,"token":pushover_api_key,"message":message}

    requests.post(pushover_site_url,data=payload)

## Agent Tools

(Tools desc )

In [8]:
def record_user_details(email, name="Name not provided",notes="not provided"):
    """Tool to record user details """
    pushover_notification(f"Recording user details for {name} with email {email} and notes {notes}")
    return {"recorded": "Ok"}

In [11]:
def record_unknown_question(question):
    """This will send a pushover notification to us , notifying us that the agent
    was asked an unknown question"""
    pushover_notification(f"Recording {question}, I could not answer it")
    return {"recorded": "Ok"}


In [ ]:
#The following code is boiler plate code for the agent, it's not something you'll typically use
#since frameworks already have this built in

#The LLM will use this to decide wether it's appropiate or not to use this tool
record_user_details_json={
    "name": "record_user_details",
    "description": "Use this tool to record everytime a user is interested in being in touch and provided an email address",
    "parameters": {
        "type":"object",
        "properties": {
            "email": {

                "type": "string",

                "description": "the email address of the user"
            },
            "name": {

                "type": "string",

                "description": "the user's name, if they provided it"
            },
            "notes": {

                "type": "string",

                "description": "Any additional notes , information or such  about the conversation the user provided that's worth recording"

            }
        },
        "required": ["email"],
        "additionalProperties": False
        
        }
}


In [ ]:
#A list with the tools that the agent can use
tools=[{"type":"function","function":record_user_details},
{"type":"function","function":record_unknown_question}]